In [1]:
!pip -V

pip 21.0.1 from /Users/cescqi/PycharmProjects/venvCollection/phbs/lib/python3.7/site-packages/pip (python 3.7)


### 1. Introduction

We suppose that the price at time t,$S_t$, is given by
$$
S_t = S_0exp(\sigma B_t-\frac{1}{2}\sigma^2t+ct)
$$
where B is a standard one-dimensional Brownian motion, $c=r$ is riskless interest rate.

We want to calculate
$$
E(Y-K)^+
$$
where we define Y by
$$
Y\equiv \int_0^T S_u\mu (du)
$$

similarly, for a Discretely monitored Asian option, we have
$$
C(T)=(\sum_{k=1}^N w_kS_k(t_k) - K)^+
$$
where, $w_k=1/N,\ 0\le t_1<...<t_N=T$ and $S_k(t)$s are identical; $N\gg10$.

**Let's give a Python example with fixed strike.**

In [2]:
import numpy as np

def simulate_single_price__path(spot, sigma, nsteps, texp, intr=0.0):
    '''generate single simulated path of price of assets'''
    
    if isinstance(sigma, (int, float)):
        sigma = [sigma] * nsteps
        
    dt = texp/nsteps
    sqdt = np.sqrt(dt)
    log_price = np.log(spot)
    log_price_path = np.empty(nsteps)
    log_price_path[0] = log_price
    
    wt = np.random.normal(0, 1, nsteps)
    for i in range(1, nsteps):
        log_price_path[i] = log_price_path[i-1] + sigma[i]*sqdt*wt[i]-0.5*sigma[i]**2*dt + intr*dt
    
    price_path = np.exp(log_price_path)
    
    return price_path

def simulate_multiple_price_path(spot, sigma, nsteps, texp, npath, intr=0.0):
    '''generate multiple paths of asset price'''
    
    prices_path = np.empty((npath, nsteps))
    for i in range(npath):
        prices_path[i, :] = simulate_single_price__path(spot, sigma, nsteps, texp, intr)
        
    return prices_path


def solve_asian_option_price_mc(strike, price, cp=1):
    '''given path of price and strikes, sovle the price of an Asian option'''
    
    npath, nsteps = price.shape
    if isinstance(strike, (int, float)):
        strike = [strike] * npath
        
    price_on_path = np.empty(npath)
    for i in range(npath):
        if cp == 1:
            price_on_path[i] = np.fmax(np.mean(price[i, :])-strike[i], 0)
        else:
            price_on_path[i] = np.fmax(strike[i]-np.mean(price[i, :]), 0)
    
    
    return np.mean(price_on_path)

In [3]:
spot = 100
sigma = 0.05
nsteps = 30
texp = 1
intr = 0.05

price_path = simulate_single_price__path(spot, sigma, nsteps, texp, intr)
price_path

array([100.        ,  99.53781596, 100.04173781, 100.27891706,
       101.22916525, 101.61724256, 101.19097114, 101.43998789,
       102.61424011, 103.33202649, 101.64724273, 100.970429  ,
       101.70358691, 102.12139342, 102.82504078, 105.79921616,
       106.06045077, 104.80265891, 103.97098766, 105.28611347,
       104.80793555, 104.27599282, 104.47284298, 107.32196278,
       106.12157635, 106.84197729, 108.44769961, 108.17220381,
       106.98153426, 107.7664782 ])

In [4]:
npath = 10000

prices_path = simulate_multiple_price_path(spot, sigma, nsteps, texp, npath, intr)
prices_path

array([[100.        , 101.93331326, 102.17213891, ..., 107.67333394,
        106.63995158, 106.27033391],
       [100.        ,  99.99152873,  99.33716646, ..., 105.9977183 ,
        105.77316316, 106.85238147],
       [100.        ,  98.66663376,  98.02472512, ..., 100.14569927,
         99.30133907,  99.95497249],
       ...,
       [100.        , 101.06686221, 101.48440376, ..., 105.76398524,
        105.59881707, 104.81118926],
       [100.        ,  99.61680235, 101.22445305, ..., 101.88584558,
        100.51205788, 102.29124677],
       [100.        ,  99.79984946, 100.92218647, ..., 107.94123552,
        108.83250333, 109.07788296]])

### MC result of option price

In [6]:
strike = 100

asian_option_price_fixed = solve_asian_option_price_mc(strike, prices_path)
print(f'The Asian option price with fixed strike is {asian_option_price_fixed}')

The Asian option price with fixed strike is 2.8028620544557636


In [14]:
sigma = 0.1
intr = 0.05
prices_path = simulate_multiple_price_path(spot, sigma, nsteps, texp, npath, intr)

strike = prices_path[:, nsteps-1]

asian_option_price_floating = solve_asian_option_price_mc(strike, prices_path)
print(f'The Asian option price with floating strike is {asian_option_price_floating}')

The Asian option price with floating strike is 1.2455038235427935


These two results are similar to the paper.

### Validation
An Asian option price is usually lower than European or American, because of lower volatility, so let's validate that.

In [8]:
import pyfeng as pf

strike = 100
euro_option_price = pf.Bsm.price_formula(strike, spot, sigma, texp)
print(f'The European option price is {euro_option_price}')

The European option price is 1.9945036390476076


### 2. PDE for the price of an Asian option

for a fixed strike option, we have
$$
e^{-rT}E(\int_0^T(S_u-K)\frac{du}{T})^+=S_0f(0,KS_0^{-1}) \\
\equiv e^{-rT}S_0\phi(0, KS_0^{-1})
$$

for a floating strike option, we have
$$
e^{-rT}E(\int_0^TS_u\frac{du}{T}-S_T)^+ = e^{-rT}S_0\psi(0,0)
$$

where 

$\phi(t,x)=r^{-1}(e^{r(T-t)-1}-x),$ and $(x\le0)$;

$\psi(t,x)=\frac{e^{r(T-t)}-1}{rT}-e^{r(T-t)}-x$, and $x$ is large negative;

There should be a numerical method to solve this using *NAG D03PAF*.

### 3. Lower Bound

#### Definition

To obtain a lower bound, we have
$$
E(Y^+)=E(E(Y^+|Z))\ge E(E(Y|Z)^+)
$$
the right side is the lower bound.

#### Compute the Mean

Let's first simplify the question to $S_0=1$.

Firstly we need to compute the mean of $Y$, this is given by
$$
E[Y|Z]=\int_0^1exp(\sigma m_tZ-\frac{1}{2}\sigma^2\nu m_t^2+rt)u(dt)
$$
where we let $Z=\int_0^1B_tdt$, thus $m_t=3t(2-t)/2$, and $\nu \equiv Var(Z)=\frac{1}{3}$

#### Compute the Variance

Based on the mean expression, we can obtain the variance finally
$$
var(Y|Z) \le I_1 + I_2
$$
where $I_1, I_2$ has long expressions which we do not display here.

we turn to the estimates
$$
0\le E[E(Y^+|Z]-E[Y|Z]^+]\le \frac{1}{2}E(I_1+I_2)^{1/2}
$$

we can compute the right side of above is
$$
\frac{1}{2}[c\sigma^2exp(c\sigma^2+\gamma_2)(\frac{1}{2}\sigma^2\gamma_1\nu exp(\frac{1}{2}\sigma^2\gamma_1\nu)+\frac{1}{2}\gamma_2^2)+\frac{1}{2}\sigma^4c^2exp(\sigma^4c^2)(1+\gamma_2)]^{1/2}M^{1/2}
$$

for fixed strike, we may take that:

$\nu=1/3$

$\gamma_1=9$

$\gamma_2=r+\sigma^2/4$

$c=1/3$

$M=1$

for floating strike, the differences are that

$c=3^{2/3}/4$

$M=4$

#### Summary
The mean is to compute the lower bound and the variance is helpful to compute the upper bound.